In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.optimizers import Adam


from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# https://github.com/eriklindernoren/Keras-GAN
import time
import os
import csv
from PIL import Image

root = 'CGAN_results_extra/'

# plotting data structure
train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []
train_hist['accuracy'] = []
train_hist['Model'] = []
train_hist['classif_loss'] = []
train_hist['classif_acc'] = []

Using TensorFlow backend.


In [2]:
def trainClassif():
    (train_set, train_label_raw), (_, _) = mnist.load_data()
    train_label = to_categorical(train_label_raw)
    train_data = np.asarray(train_set).reshape(60000, 28, 28, 1)
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.33, random_state=43)

#     test_data = test_set.reshape(10000, 28, 28, 1)

#     np.shape(train_data)

    model = Sequential()
    # model.add(Conv2D(5, kernel_size=3, activation='relu', input_shape=(28,28,1)))
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # model.summary()

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=8)
    return model

def getClassif():
    model = Sequential()
    # model.add(Conv2D(5, kernel_size=3, activation='relu', input_shape=(28,28,1)))
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model
                       

In [3]:
# classif = trainClassif()
# os.makedirs('saved_model_weights/classif/', exist_ok=True)
# classif.save_weights('saved_model_weights/classif/pre_trained_classif_short.h5')

In [4]:
classif = getClassif()
classif.load_weights('saved_model_weights/classif/pre_trained_classif.h5', by_name=False)


In [ ]:
(_,_), (X_test_raw, y_test_raw) = mnist.load_data()
X_test = X_test_raw.reshape(10000,28,28,1)
y_test = to_categorical(y_test_raw)
classif.evaluate(X_test, y_test)

In [ ]:
class CGAN():
    def __init__(self, pick_model='Deep_BN'):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator(pick_model=pick_model)
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator(pick_model=pick_model)

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self, pick_model='Deep_BN'):
        if (pick_model == 'Deep_BN') | (pick_model == 'Deeper_G_BN'):
            model = Sequential()

            model.add(Dense(256, input_dim=self.latent_dim))
            model.add(LeakyReLU(alpha=0.2))
            model.add(BatchNormalization(momentum=0.8))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(BatchNormalization(momentum=0.8))
            model.add(Dense(1024))
            model.add(LeakyReLU(alpha=0.2))
            model.add(BatchNormalization(momentum=0.8))
            model.add(Dense(np.prod(self.img_shape), activation='tanh'))
            model.add(Reshape(self.img_shape))

            model.summary()

        elif (pick_model == 'Shallow_BN') | (pick_model == 'Deeper_D_BN'): 
            model = Sequential()

            model.add(Dense(256, input_dim=self.latent_dim))
            model.add(LeakyReLU(alpha=0.2))
            model.add(BatchNormalization(momentum=0.8))
            model.add(Dense(np.prod(self.img_shape), activation='tanh'))
            model.add(Reshape(self.img_shape))

            model.summary()
        elif pick_model == 'Shallow_Drop':
            model = Sequential()

            model.add(Dense(256, input_dim=self.latent_dim))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dropout(0.4))
            model.add(Dense(np.prod(self.img_shape), activation='tanh'))
            model.add(Reshape(self.img_shape))

            model.summary()
        elif pick_model == 'Drop':
            model = Sequential()

            model.add(Dense(256, input_dim=self.latent_dim))
            model.add(LeakyReLU(alpha=0.2))
            model.add(BatchNormalization(momentum=0.8))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(BatchNormalization(momentum=0.8))
            model.add(Dense(1024))
            model.add(LeakyReLU(alpha=0.2))
            model.add(BatchNormalization(momentum=0.8))
            model.add(Dense(np.prod(self.img_shape), activation='tanh'))
            model.add(Reshape(self.img_shape))

            model.summary()
        elif pick_model == 'Shallow_simple':
            model = Sequential()

            model.add(Dense(128, input_dim=self.latent_dim))
            model.add(LeakyReLU(alpha=0.2))
#             model.add(BatchNormalization(momentum=0.8))
            model.add(Dense(np.prod(self.img_shape), activation='tanh'))
            model.add(Reshape(self.img_shape))

            model.summary()
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self, pick_model='Deep_BN'):
        if (pick_model == 'Deep_BN') | (pick_model == 'Deeper_D_BN'):
            model = Sequential()

            model.add(Dense(512, input_dim=np.prod(self.img_shape)))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dropout(0.4))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dropout(0.4))
            model.add(Dense(1, activation='sigmoid'))
            model.summary()
        
        elif (pick_model == 'Shallow_BN') | (pick_model == 'Deeper_G_BN'): 
            model = Sequential()

            model.add(Dense(512, input_dim=np.prod(self.img_shape)))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dropout(0.4))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dropout(0.4))
            model.add(Dense(1, activation='sigmoid'))
            model.summary()
        
        elif pick_model == 'Shallow_Drop':
            model = Sequential()

            model.add(Dense(512, input_dim=np.prod(self.img_shape)))
            model.add(LeakyReLU(alpha=0.2))

            model.add(Dropout(0.4))
            model.add(Dense(1, activation='sigmoid'))
            model.summary()
        
        elif pick_model == 'Drop':
            model = Sequential()

            model.add(Dense(512, input_dim=np.prod(self.img_shape)))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dropout(0.4))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dropout(0.4))
            model.add(Dense(1, activation='sigmoid'))
            model.summary()
            
        elif pick_model == 'Shallow_simple':
            model = Sequential()

            model.add(Dense(128, input_dim=np.prod(self.img_shape)))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dense(1, activation='sigmoid'))
            model.summary()      
            
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50, ratio = (1,1), pick_model = 'Deep_BN'):
        best_acc = 0
        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------
            epoch_start_time = time.time()
            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])
            #implement ratio of D  
            if epoch % ratio[0] == 0:
                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)
            #implement ratio of G
            if epoch % ratio[1] == 0:
                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            # save data for plotting and csv
            train_hist['D_losses'].append(d_loss[0])
            train_hist['G_losses'].append(g_loss)
            train_hist['per_epoch_ptimes'].append(per_epoch_ptime)    
            train_hist['accuracy'].append(100 * d_loss[1])
            train_hist['Model'].append(pick_model)
            
            

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch, pick_model, ratio)
                # Plot the progress
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                # REMEMBER TO CHANGE TO PARAM DIRECTORY
                if pick_model == 'Deep_BN':
                    gen_images, gen_labels = cgan.generate_images(100)
                    loss, acc = classif.evaluate(gen_images, to_categorical(gen_labels))
                    train_hist['classif_loss'].append(loss)
                    train_hist['classif_acc'].append(acc)
                    
                    if (acc > 0.925) & (acc > best_acc):
                        best_acc = acc
                        
                        gen_images, gen_labels = cgan.generate_images(10000)
                        image_out = gen_images.reshape(100000,784)

                        df = pd.DataFrame(image_out)
                        df.to_csv('generated_images_%d'%epoch +'_%f.csv'%acc, index=False, header=False)
                        dfl = pd.DataFrame(gen_labels)
                        dfl.to_csv('generated_labels_%d'%epoch +'_%f.csv'%acc, index=False, header=False)
#                 os.makedirs('saved_model_weights/generator_weights/', exist_ok=True)
#                 generator.save_weights('saved_model_weights/generator_weights/' + pick_model + '_' + ratio + '_' + epoch + '.h5')

#             if epoch == range(epochs):    
#                 os.makedirs('saved_model_weights/generator_weights/', exist_ok=True)
#                 generator.save_weights('saved_model_weights/generator_weights/' + pick_model + '_' + ratio + '_' + epoch + '.h5')

#                 os.makedirs('saved_model_weights', exist_ok=True)
#                 self.generator.save_weights('saved_model_weights/generator_weights_%d.h5'%epoch)
#                 discriminator.save_weights('saved_model_weights/discriminator_weights_%d.h5'%epoch)
#                 combined.save_weights('saved_model_weights/combined_weights-%d.h5'%epoch)

    def sample_images(self, epoch, pick_model, ratio):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)

        gen_imgs = self.generator.predict([noise, sampled_labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
                axs[i,j].axis('off')
                cnt += 1
        os.makedirs(root + 'images/'+  pick_model + '/'+ str(ratio[0]) + '_' + str(ratio[1]) + '/' , exist_ok=True)
        fig.savefig(root + 'images/'+  pick_model + '/'+ str(ratio[0]) + '_' + str(ratio[1]) + '/' + str(epoch) + '.png')
#         fig.savefig("images/%d.png" % epoch)
        plt.close()
        
    def generate_images(self, num):
        r, c = 2, 5
        arr_imgs = []
        arr_labels = []
        for i in range(0, num):
            noise = np.random.normal(0, 1, (2 * 5, 100))
            sampled_labels = np.arange(0, 10).reshape(-1, 1)
            gen_imgs = self.generator.predict([noise, sampled_labels])

            # Rescale images 0 - 1
            gen_imgs = 0.5 * gen_imgs + 0.5
            gen_images = ((gen_imgs.astype(np.float32) * 255)) 

            arr_imgs.append(gen_images)
            arr_labels.append(sampled_labels)
            
        out_imgs = np.asarray(arr_imgs).reshape(num*10, 28, 28, 1)
        out_labels = np.asarray(arr_labels).reshape(num*10, 1)
        
        return out_imgs, out_labels

In [ ]:
def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))
    
    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()
    
    
    if save:
        
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

In [ ]:
ratio_array = [ (1,1), (1,2), (1,3),  ]
# ratio_array = [ (1,1),(2,1), (1,2), (3,1), (1,3), (5,1), (1,5), (1,10), (10,1)]

# model_array = ['Shallow_simple', 'Shallow_BN'  ]
model_array = ['Deep_BN' ]

# model_array = ['Deep_BN' 'Deeper_G_BN']


for pick_model in model_array:
    if pick_model != model_array[0]:
        del cgan
    #---------------------------COMPILE SELECTED MODELS--------------------------------------#
    #----------------------------------------LOOP OVER RATIOS--------------------------------#
    for ratio in ratio_array:
        if ratio != ratio_array[0]:
            del cgan
            
        cgan = CGAN(pick_model=pick_model)

         #----------------------------------------EXECUTION-----------------------------------#
        start = time.time()

        cgan.train(epochs=50001, batch_size=64, sample_interval=100, ratio=ratio, pick_model=pick_model) ## ratio G:D
        
        end = time.time()
        
        elapsed_train_time = 'elapsed training time: {} min, {} sec '.format(int((end - start) / 60),
                                                                             int((end - start) % 60))
        train_hist['total_ptime'].append(elapsed_train_time)

        print(elapsed_train_time)
        os.makedirs(root + 'hist/', exist_ok=True)  
        show_train_hist(train_hist, save=True, path=root + 'hist/' + str(ratio[0]) + '_' + str(ratio[1]) + pick_model +'.png')
        # save hist data to csv
        gen_images, gen_labels = cgan.generate_images(1000)
        loss, acc = classif.evaluate(gen_images, to_categorical(gen_labels))
        train_hist['classif_loss'].append(loss)
        train_hist['classif_acc'].append(acc)
        
        os.makedirs(root + 'hist_csv/', exist_ok=True) 
        with open(root+ 'hist_csv/' + str(ratio[0]) + '_' + str(ratio[1]) + pick_model +'.csv', 'w') as f:
            for key in train_hist.keys():
                f.write("%s,%s\n"%(key,train_hist[key]))
        
        
        # remove old data structure
        del train_hist
        # redefine
        train_hist = {}
        train_hist['D_losses'] = []
        train_hist['G_losses'] = []
        train_hist['per_epoch_ptimes'] = []
        train_hist['total_ptime'] = []
        train_hist['accuracy'] = []
        train_hist['Model'] = []
        train_hist['classif_loss'] = []
        train_hist['classif_acc'] = []
        gen_images, gen_labels = cgan.generate_images(1000)

        loss, acc = classif.evaluate(gen_images, to_categorical(gen_labels))
        
#         os.makedirs('saved_model_weights/generator_weights/', exist_ok=True)
#         os.makedirs('saved_model_weights/discriminator_weights/', exist_ok=True)
#         os.makedirs('saved_model_weights/combined_weights/', exist_ok=True)
#         generator.save_weights('saved_model_weights/generator_weights/' + pick_model + '_' + ratio + '_' + epoch + '.h5')
#         discriminator.save_weights('saved_model_weights/discriminator_weights/' + pick_model + '_' + ratio + '_' + epoch + '.h5')
#         combined.save_weights('saved_model_weights/combined_weights-%d.h5' + pick_model + '_' + ratio + '_' + epoch + '.h5')
       
        

In [9]:
# print(gen_images.shape)
# image_out = gen_images.reshape(10000,784)
# print(image_out.shape)

In [10]:
# df = pd.DataFrame(image_out)
# df.to_csv('generated_images.csv',index=False, header=False)
# dfl = pd.DataFrame(gen_labels)
# dfl.to_csv('generated_labels.csv',index=False, header=False)
# epoch = 0
# acc = 0.5
# gen_images, gen_labels = cgan.generate_images(10000)
# image_out = gen_images.reshape(100000,784)

In [11]:
# df = pd.DataFrame(image_out)
# df.to_csv('generated_images_%d'%epoch +'_%f.csv'%acc, index=False, header=False)
# dfl = pd.DataFrame(gen_labels)
# dfl.to_csv('generated_labels_%d'%epoch +'_%f.csv'%acc, index=False, header=False)

In [12]:
# gen_images_raw, gen_labels = cgan.generate_images(1000)
# # gen_imgs = 0.5 * gen_images + 0.5
# gen_images = ((gen_images_raw.astype(np.float32) * 255)) 

# gen_labels_cat = to_categorical(gen_labels)
# loss, acc = classif.evaluate(gen_images, gen_labels_cat)
# print(loss, acc)

In [13]:
# print(gen_images[0])
# print(X_test[0])


In [14]:
# print(gen_labels_cat[9])
# print(y_test[0])

In [15]:
# print(classif.evaluate(X_test[0:5], y_test[0:5]))
# print(classif.predict(X_test[0:5]))


In [16]:
# print(loss)
# print(acc)

In [17]:
# print(X_test.shape)
# print(y_test.shape)
# print(gen_images.shape)
# print(to_categorical(gen_labels).shape)


In [18]:
# print(to_categorical(gen_labels))
# print(classif.predict(X_test))
# to_categorical(gen_labels)

In [19]:
# print(gen_labels[3])
# print(to_categorical(gen_labels[3][0]))
# print(y_test[1])

In [20]:
# print(gen_labels.shape)
# print(gen_images.shape)

# print(gen_labels[5005])
# plt.imshow(gen_images[5003,:,:, 0], cmap='gray')

# print(classif.metrics_names)